In [1]:
import torch 
import torch.nn as nn
import torchvision
from utils import ALL_LETTERS , N_LETTERS
from utils import line_to_tensor,letter_to_index,letter_to_tensor,load_data,random_training_example

import matplotlib.pyplot as plt
import sys
import time

In [2]:
start = time.time()

In [3]:
class RNN(nn.Module):
    def __init__(self,input_size , hidden_size , num_layers, num_classes):
        super(RNN,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size,hidden_size,num_layers,batch_first = True)
        self.fc = nn.Linear(hidden_size,num_classes)

    def forward(self , x ):
        h0 = torch.zeros(self.num_layers , x.size(0) ,self.hidden_size)
        out ,_ = self.rnn(x,h0)
        out = out[:,-1,:]
        out = self.fc(out)
        return out   # out_shape = [1(batch_size) , 57(seq_len) , 128(hidden_size)]

In [4]:
dict_category , all_categories = load_data()

In [5]:
input_size = N_LETTERS
hidden_size = 256
num_layers = 3
num_classes = len(all_categories)
learning_rate = 0.001
n_epoch = 1000000

In [6]:
modelRNN  = RNN(input_size,hidden_size,num_layers,num_classes)
modelRNN

RNN(
  (rnn): RNN(57, 256, num_layers=3, batch_first=True)
  (fc): Linear(in_features=256, out_features=18, bias=True)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(modelRNN.parameters(), lr=learning_rate)

In [8]:
print_step = 10000
for epoch in range(n_epoch):
    category, line, category_tensor, line_tensor = random_training_example(dict_category,all_categories)
    line_tensor = line_tensor.reshape(line_tensor.size(1),line_tensor.size(0),N_LETTERS)

    outputs = modelRNN(line_tensor)
    # print(outputs.shape)
    # print(category_tensor.shape)
    # break
    loss = criterion(outputs,category_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1)%print_step == 0:
        print(f'[{(epoch+1)/print_step}/{n_epoch/print_step}], loss = {loss.item() :.4f}')

[1.0/100.0], loss = 1.8091
[2.0/100.0], loss = 2.7682
[3.0/100.0], loss = 1.2032
[4.0/100.0], loss = 2.4664
[5.0/100.0], loss = 2.1895
[6.0/100.0], loss = 0.9668
[7.0/100.0], loss = 0.8901
[8.0/100.0], loss = 1.3835
[9.0/100.0], loss = 0.5712
[10.0/100.0], loss = 0.0609
[11.0/100.0], loss = 1.9016
[12.0/100.0], loss = 1.2974
[13.0/100.0], loss = 3.4619
[14.0/100.0], loss = 1.5074
[15.0/100.0], loss = 0.0797
[16.0/100.0], loss = 1.3681
[17.0/100.0], loss = 0.0899
[18.0/100.0], loss = 1.4266
[19.0/100.0], loss = 0.2004
[20.0/100.0], loss = 1.1627
[21.0/100.0], loss = 1.1862
[22.0/100.0], loss = 3.9355
[23.0/100.0], loss = 0.0453
[24.0/100.0], loss = 0.0480
[25.0/100.0], loss = 0.7224
[26.0/100.0], loss = 1.6602
[27.0/100.0], loss = 1.5872
[28.0/100.0], loss = 0.2235
[29.0/100.0], loss = 0.2748
[30.0/100.0], loss = 0.0293
[31.0/100.0], loss = 0.0222
[32.0/100.0], loss = 0.0039
[33.0/100.0], loss = 0.1830
[34.0/100.0], loss = 0.0406
[35.0/100.0], loss = 0.0278
[36.0/100.0], loss = 0.0013
[

In [9]:
def prediction(output):
    my_pred_idx = torch.argmax(output)
    return all_categories[my_pred_idx]

In [16]:
with torch.no_grad():
    n_correct = 0
    n_sampes = 0
    for i in range(20000):
        category, line, category_tensor, line_tensor = random_training_example(dict_category,all_categories)
        line_tensor = line_tensor.reshape(line_tensor.size(1),line_tensor.size(0),N_LETTERS)
        
        output = modelRNN(line_tensor)

        my_pred = prediction(output)
        if my_pred == category:
            n_correct+=1

    print(f'Accuracy = {100*(n_correct/20000)}')

Accuracy = 94.58


In [11]:
end = time.time()
print(f'time = {(end-start)/60} min')

time = 89.75921707948049 min


In [12]:
torch.save(modelRNN.state_dict(),'again_model.pth')